# Convolutional Neural Networks for Image Segmentation

**This notebook shows how to create a CNN model for image segmentation with TensorFlow 2 and Keras**

## Table of Contents
- ### [Python libraries](#libraries)